In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.data import Field, TabularDataset, BucketIterator

from dataset.mtgcards import RuleText
from utils.preprocess import fields_for_rule_text

import random
import math
import time
import os

d:\Miniconda3\envs\seq2seq\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SRC, TRG = fields_for_rule_text()
fields = {'src': ('src', SRC), 'trg': ('trg', TRG)}

train_data, valid_data, test_data = RuleText.splits(fields=fields, version='v2.1')

In [3]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)
print(f"Unique tokens in source (en) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (zh) vocabulary: {len(TRG.vocab)}")

for x in random.sample(list(train_data), 3):
    print(x.src, x.trg)

Unique tokens in source (en) vocabulary: 1475
Unique tokens in target (zh) vocabulary: 2306
['this', 'effect', 'reduces', 'only', 'the', 'amount', 'of', 'white', 'mana', 'you', 'pay', '.'] ['此', '效应', '仅', '减少', '你', '支付', '的', '白色', '法术力', '数量', '。']
['{', '2', '}', '{', 'r', '}', ':', '<', '8', '>', 'gets', '+', '2', '/', '+', '0', 'until', 'end', 'of', 'turn', '.'] ['{', '2', '}', '{', 'r', '}', '：', '<', '8', '>', '得', '+', '2', '/', '+', '0', '直到', '回合', '结束', '。']
['then', 'that', 'player', 'shuffles', '.'] ['然后', '该', '牌手', '将', '其', '牌库', '洗牌', '。']


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    sort_within_batch = True,
    sort_key = lambda x: len(x.src),
    device = device)

tmp = next(iter(train_iterator))
print(tmp)

cpu

[torchtext.legacy.data.batch.Batch of size 128]
	[.src]:('[torch.LongTensor of size 6x128]', '[torch.LongTensor of size 128]')
	[.trg]:[torch.LongTensor of size 8x128]


In [5]:
from models.model4.definition import Encoder, Attention, Decoder, Seq2Seq
from models.model4.train import init_weights, train, evaluate
from utils import count_parameters, train_loop

In [6]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)

model.apply(init_weights)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 11,535,874 trainable parameters


In [7]:
optimizer = optim.Adam(model.parameters())
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

train_loop(model, optimizer, criterion, train, evaluate,
           train_iterator, valid_iterator, 
           save_path='result/', file_name='model4-rule-v2.1.pt', load_before_train=True)

model will be saved to result/model4-rule-v2.1.pt
load model parameters from result/model4-rule-v2.1.pt
Val. Loss: 1.203 |  Val. PPL:   3.331


  0%|          | 0/291 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [7]:
from utils.translate import Translator
from models.model4.definition import beam_search
model.load_state_dict(torch.load('result/model4-rule-v2.1.pt', map_location=torch.device(device)))
T = Translator(SRC, TRG, model, device, beam_search)
torch.save(T,'result/model4-T-v2.1.pt')

In [8]:
data = 'Whenever <1> becomes attached to a creature, for as long as <1> remains attached to it, you may have that creature become a copy of another target creature you control.'
data = 'target creature gets - 1 / - 1 until end of turn .'
ret, prob = T.translate(data, max_len=100)
print(*ret[:3], sep='\n')

['目标', '生物', '得', '-', '1', '/', '-', '1', '直到', '回合', '结束', '。', '<eos>']
['目标', '生物', '得', '得', '-', '1', '/', '-', '1', '直到', '回合', '结束', '。', '<eos>']
['目标', '生物', '得', '1', '/', '-', '1', '直到', '回合', '结束', '。', '<eos>']


In [9]:
from utils import show_samples
long_data = [x for x in test_data.examples if len(x.src) > 20]
print(f'Number of samples: {len(long_data)}')
show_samples(long_data, T, n=100, beam_size=3)

Number of samples: 212
src: [whenever < 1 > attacks , other humans you control get + 1 / + 0 until end of turn . ] trg = [每当<1>攻击时，由你操控的其他人类得+1/+0直到回合结束。]
每当<1>攻击时，由你操控的其他人类得+1/+0直到回合结束。<eos> 	[probability: 0.60869]
每当<1>攻击时，由你操控的其他人类生物得+1/+0直到回合结束。<eos> 	[probability: 0.06468]
每当<1>攻击时，其他由你操控的其他人类得+1/+0直到回合结束。<eos> 	[probability: 0.03871]

src: [channel — { x } { x } { g } { g } , discard shigeki : return x target non legendary cards from your graveyard to your hand . ] trg = [魂力～{x}{x}{g}{g}，弃掉茂树：将x张目标非传奇的牌从你坟墓场移回你手上。]
魂力～{x}{x}{g}{g}，弃掉茂树：将x张目标非传奇牌从你的坟墓场移回你手上。<eos> 	[probability: 0.02929]
魂力～{x}{x}{g}{g}，弃掉茂树：将x张目标非传奇永久物牌从你的坟墓场移回你手上。<eos> 	[probability: 0.02464]
魂力～{x}{x}{g}{g}，弃掉茂树：将x个目标非传奇牌从你的坟墓场移回你手上。<eos> 	[probability: 0.00961]

src: [{ 1 } , { t } , sacrifice < 7 > : return target instant or sorcery card from your graveyard to your hand . ] trg = [{1}，{t}，牺牲<7>：将目标瞬间或法术牌从你的坟墓场移回你手上。]
{1}，{t}，牺牲<7>：将目标瞬间或法术牌从你的坟墓场移回你手上。<eos> 	[probability: 0.74368]
{1}，{t}，牺牲<7>：将目标瞬间或从你的坟墓场移回你

KeyboardInterrupt: 

In [10]:
from dataset.mtgcards import TestSets
from utils import calculate_bleu
from torchtext.legacy.data import Field
from models.card_name_detector.definition import TrainedDetector
from utils.translate import sentencize, CardTranslator

fields = {'src-rule': ('src', Field(tokenize=lambda x: x.split(' '))), 'trg-rule': ('trg', Field())}
test_data = TestSets.load(fields)

D = TrainedDetector()

path: d:\Desktop\mtg-cards-translation\models\card_name_detector


In [11]:
def sentencize(text: str):
    ignore = {' ', '(', ')', '\n'}
    while len(text) and text[0] in ignore:
        text = text[1:]
    if len(text) == 0:
        return []
    
    r = 0
    delims = {'.', '\n', '('}
    ignore = False
    while r < len(text):
        if text[r] == '\"':
            ignore = not ignore
        if not ignore and text[r] in delims:
            break
        r += 1
    
    if r < len(text) and text[r] == '.':
        return [text[:r + 1]] + sentencize(text[r + 1:])
    return [text[:r]] + sentencize(text[r:])
def preprocess(x:str):
    x = D.annotate(x).removeprefix(' ')
    print(f'[after preprocess]:{x}')
    return x
def postprocess(x:str):
    return x.replace('<', '').replace('>', '')

import re
class CTHelper:
    def __init__(self, name_detector, dictionary={}) -> None:
        self.D = name_detector
        self.dictionary = dictionary
    
    def preprocess(self, x:str):
        self.tag2str = {}
        x = D.annotate(x).removeprefix(' ') # x become lowercase after go through detector
        m = re.search('<[^0-9>]+>', x)
        id = 0
        while m:
            l, r = m.span()
            tag = '<' + str(id) + '>'
            self.tag2str[tag] = x[l:r]
            x = x[:l] + tag + x[r:]
            id += 1
            m = re.search('<[^0-9>]+>', x)

        for s in self.dictionary.keys():
            m = re.search(s, x)
            if m:
                tag = '<' + str(id) + '>'
                self.tag2str[tag] = s
                x = x.replace(s, tag)
                id += 1

        print(f'[after preprocess]:{x}')
        return x

    def postprocess(self, x:str):
        print(f'[before postprocess]:{x}')
        for tag, s in self.tag2str.items():
            x = x.replace(tag, self.dictionary[s] if s in self.dictionary else s)
        return x

dic = {}
dic = {'oil':'烁油', 'rebel':'反抗军','compleated':'完化'}
helper = CTHelper(D, dic)
CT = CardTranslator(sentencize, T, preprocess=lambda x: helper.preprocess(x), postprocess=lambda x:helper.postprocess(x))

# example = random.sample(list(test_data), 1)[0]
T=0
for example in list(test_data)[T:T+50]:
    print(vars(example))
    CT.translate(' '.join(example.src))


example = list(test_data)[15]
print(vars(example))
CT.translate(' '.join(example.src))
# for example in random.sample(list(test_data), 10):
#     print(vars(example))
#     CT.translate(' '.join(example.src))

{'src': ['Whenever', 'another', 'artifact', 'or', 'creature', 'you', 'control', 'is', 'put', 'into', 'a', 'graveyard', 'from', 'the', 'battlefield,', 'put', 'an', 'oil', 'counter', 'on', 'Vat', 'of', 'Rebirth.\n{2}{B},', '{T},', 'Remove', 'four', 'oil', 'counters', 'from', 'Vat', 'of', 'Rebirth:', 'Return', 'target', 'creature', 'card', 'from', 'your', 'graveyard', 'to', 'the', 'battlefield.', 'Activate', 'only', 'as', 'a', 'sorcery.'], 'trg': ['每当另一个由你操控的神器或生物从战场进入坟墓场时，在再生缸上放置一个烁油指示物。', '{2}{B}，{T}，从再生缸上移去四个烁油指示物：将目标生物牌从你的坟墓场移回战场。只能于法术时机起动。']}
[after preprocess]:whenever another artifact or creature you control is put into a graveyard from the battlefield , put an <1> counter on <0> .
[before postprocess]:每当另一个由你操控的神器或生物从场上进入坟墓场时，在<0>上放置一个<1>。
[after preprocess]:{2} {b} , {t} , remove four <1> counters from <0> : return target creature card from your graveyard to the battlefield .
[before postprocess]:{2}{b}，{t}，从<0>上移去四个充电指示物物：将目标生物牌从你的坟墓场移回战场。
[after preprocess]:activate only as a s

'飞行 <necrosquito>进战场时上面有两个+1指示物。 烁油上每有一个1指示物，<necrosquito>便得+1/+1。 每当另一个由你操控的生物或鹏洛客从战场进入坟墓场时，在<necrosquito>上放置一个烁油。'

In [12]:
from utils import calculate_testset_bleu
calculate_testset_bleu(list(test_data)[:100], CT)

  0%|          | 0/100 [00:00<?, ?it/s]

[after preprocess]:whenever another artifact or creature you control is put into a graveyard from the battlefield , put an <1> counter on <0> .
[before postprocess]:每当另一个由你操控的神器或生物从场上进入坟墓场时，在<0>上放置一个<1>。
[after preprocess]:{2} {b} , {t} , remove four <1> counters from <0> : return target creature card from your graveyard to the battlefield .


  1%|          | 1/100 [00:00<00:57,  1.71it/s]

[before postprocess]:{2}{b}，{t}，从<0>上移去四个充电指示物物：将目标生物牌从你的坟墓场移回战场。
[after preprocess]:activate only as a sorcery .
[before postprocess]:只能于法术时机起动。
[after preprocess]:vigilance
[before postprocess]:警戒
[after preprocess]:<0> enters the battlefield with an <1> counter on it .
[before postprocess]:<0>进战场时上面有一个1>指示物。
[after preprocess]:remove an <1> counter from <0> : it becomes an artifact creature until end of turn .
[before postprocess]:从<0>上移去一个<9>指示物：<3>成为神器生物直到回合结束。
[after preprocess]:crew 3
[before postprocess]:搭载3
[after preprocess]:tap any number of creatures you control with total power 3 or more : this vehicle becomes an artifact creature until end of turn .


  2%|▏         | 2/100 [00:01<00:52,  1.86it/s]

...

 99%|█████████▉| 99/100 [00:40<00:00,  2.64it/s]

[before postprocess]:{1}{g}，{t}，牺牲<0>：抓一张牌。
[after preprocess]:choose one —
[before postprocess]:选择一项～
[after preprocess]:• each opponent sacrifices a non token creature .
[before postprocess]:•每位对手各牺牲一个非衍生物的生物。
[after preprocess]:• each opponent sacrifices a creature token .
[before postprocess]:•每位对手各牺牲一个生物生物。


100%|██████████| 100/100 [00:40<00:00,  2.45it/s]

[after preprocess]:• each opponent sacrifices a planeswalker .
[before postprocess]:•每位对手各牺牲一个鹏洛客。


0.6496684107267375